In [1]:
import numpy as np
import tensorflow as tf
from tqdm.auto import tqdm

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
def compute_pairwise_similairity(x, y):
    x_norms = tf.linalg.norm(x, axis=-1)
    y_norms = tf.linalg.norm(y, axis=-1)
    x_norms = tf.expand_dims(x_norms, axis=1)
    y_norms = tf.expand_dims(y_norms, axis=0)
    norms = tf.math.maximum(x_norms * y_norms, 1e-8)
    x  = tf.expand_dims(x, axis=1)
    y  = tf.expand_dims(y, axis=0)
    cossim = tf.reduce_sum(x*y, axis=-1) / norms
    return cossim

## Comparing with no Rotation 1 sample of each image (training set features), and outcommenting the y_train

In [5]:
(_,y_train),(_,y_test) = tf.keras.datasets.mnist.load_data()
data = np.load('all_MNIST_test_imgs_12samples_30deg_relu_activations.npz')
data_train = np.load('all_MNIST_train_imgs_1samples_noROT_relu_activations.npz')['arr_0']


shape = data['arr_0'].shape
X = data['arr_0']

# please modify this with
# the not rotated features
# of the training (64) before
# the relu and comment out
# y_train
X_train = data_train.reshape(-1, 64)
#y_train = np.repeat(y_test[None], 12, axis=0).ravel()

batch_size = 256
n_batches  = int(np.ceil(shape[1] / batch_size))
acc = []
pbar = tqdm(range(shape[0]))
for rot in pbar:
    for b in range(n_batches):
        li = b*batch_size
        ri = min((b+1)*batch_size, shape[1])
        dst = compute_pairwise_similairity(X[rot][li:ri], X_train)
        dst = dst.numpy() - 10.0*np.eye(*dst.shape)
        current_id = np.argmax(dst, axis=1)
        acc.append(y_train[current_id] == y_test[li:ri])
        M = np.concatenate(acc).mean()
        pbar.set_description(f'{M*100:.2f}')

M = np.concatenate(acc).mean()
print(M*100)

  0%|          | 0/12 [00:00<?, ?it/s]

75.47166666666666
